In [3]:
from segmentation_models import Unet, Nestnet, Xnet


In [4]:
import numpy as np
import skimage.measure
# import pydicom
import os
import matplotlib.pyplot as plt
import cv2
from keras import backend as K
import keras
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Input
import tensorflow as tf

In [7]:
plt.gray()
os.chdir('C:/Users/span/Documents/CNN/')

<Figure size 432x288 with 0 Axes>

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
)
        
val_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
# only 0 and 255 in seg
subfolder1 = "_0255/"
# subfolder1 = "/"

path1 = "02_data/train_image"+subfolder1
path2 = "02_data/train_mask"+subfolder1
path3 = "02_data/val_image"+subfolder1
path4 = "02_data/val_mask"+subfolder1

# # scale to 255 in val
# path1 = "02_data/train_image_re255/"
# path2 = "02_data/train_mask_re255/"
# path3 = "02_data/val_image_re255/"
# path4 = "02_data/val_mask_re255/"

batch_size1 = 16

train_image_generator = train_datagen.flow_from_directory(
    path1,
#     color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)

train_mask_generator = train_datagen.flow_from_directory(
    path2,
#     color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)

val_image_generator = val_datagen.flow_from_directory(
    path3,
#     color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)


val_mask_generator = val_datagen.flow_from_directory(
    path4,
#     color_mode = "grayscale",
    batch_size = batch_size1 #NORMALLY 4/8/16/32
)

train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

Found 472 images belonging to 1 classes.
Found 472 images belonging to 1 classes.
Found 203 images belonging to 1 classes.
Found 203 images belonging to 1 classes.


In [11]:
model = Xnet(backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose') # build UNet++
# model = Unet(backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose') # build U-Net
# model = NestNet(backbone_name='resnet50', encoder_weights='imagenet', decoder_block_type='transpose') # build DLA

model.compile('Adam', 'binary_crossentropy', ['binary_accuracy'])


AttributeError: Can't set the attribute "name", likely because it conflicts with an existing read-only @property of the object. Please choose a different name.

In [ ]:
# train model
model.fit(train_generator, val_generator)

In [ ]:
# not necessarily working
# add layer
from keras import backend as K
import keras
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Input
import tensorflow as tf

num_gpus = 2
def get_model1(optimizer, loss_metric, metrics, lr=1e-3):
    cross_tower_ops = tf.distribute.HierarchicalCopyAllReduce(num_packs=num_gpus)
    strategy = tf.distribute.MirroredStrategy(cross_device_ops=cross_tower_ops)
#     strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        inputs = keras.Input((sample_width, sample_height, 1))
    
        conv0 = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='SAME')(inputs)
        conv0 = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='SAME')(conv0)
        conv0 = tf.reshape(conv0, tf.shape(conv0))
        pool0 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv0)
        drop0 = keras.layers.Dropout(0.5)(pool0)
    
        conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(drop0)
        conv1 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(conv1)
        conv1 = tf.reshape(conv1, tf.shape(conv1))
        pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
        drop1 = keras.layers.Dropout(0.5)(pool1)

        conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(drop1)
        conv2 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(conv2)
        conv2 = tf.reshape(conv2, tf.shape(conv2))
        pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
        drop2 = keras.layers.Dropout(0.5)(pool2)

        conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(drop2)
        conv3 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(conv3)
        conv3 = tf.reshape(conv3, tf.shape(conv3))
        pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
        drop3 = keras.layers.Dropout(0.3)(pool3)

        conv4 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(drop3)
        conv4 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(conv4)
        conv4 = tf.reshape(conv4, tf.shape(conv4))
        pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
        drop4 = keras.layers.Dropout(0.3)(pool4)
        
        conv5 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='SAME')(drop4)
        conv5 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='SAME')(conv5)
        conv5 = tf.reshape(conv5, tf.shape(conv5))
        pool5 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv5)
        drop5 = keras.layers.Dropout(0.3)(pool5)

        convc5 = keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='SAME')(drop5)
        convc5 = keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='SAME')(convc5)
        convc5 = tf.reshape(convc5, tf.shape(convc5))
        
        convc5u = tf.keras.layers.Conv2DTranspose(filters=512, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(convc5)
        convc5u = tf.reshape(convc5u, tf.shape(convc5u))
        upc6 = keras.layers.concatenate([convc5u, conv5], axis=3)
        convc6 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='SAME')(upc6)
        convc6 = keras.layers.Conv2D(512, (3, 3), activation='relu', padding='SAME')(convc6)
        convc6 = tf.reshape(convc6, tf.shape(convc6))
    
        conv5u = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(convc6)
        conv5u = tf.reshape(conv5u, tf.shape(conv5u))
        up6 = keras.layers.concatenate([conv5u, conv4], axis=3)
        conv6 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(up6)
        conv6 = keras.layers.Conv2D(256, (3, 3), activation='relu', padding='SAME')(conv6)
        conv6 = tf.reshape(conv6, tf.shape(conv6))
    
        conv6u = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv6)
        conv6u = tf.reshape(conv6u, tf.shape(conv6u))
        up7 = keras.layers.concatenate([conv6u, conv3], axis=3)
        conv7 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(up7)
        conv7 = keras.layers.Conv2D(128, (3, 3), activation='relu', padding='SAME')(conv7)
        conv7 = tf.reshape(conv7, tf.shape(conv7))
    
        conv7u = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv7)
        conv7u = tf.reshape(conv7u, tf.shape(conv7u))
        up8 = keras.layers.concatenate([conv7u, conv2], axis=3)
        conv8 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(up8)
        conv8 = keras.layers.Conv2D(64, (3, 3), activation='relu', padding='SAME')(conv8)
        conv8 = tf.reshape(conv8, tf.shape(conv8))
    
        conv8u = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv8)
        conv8u = tf.reshape(conv8u, tf.shape(conv8u))
        up9 = keras.layers.concatenate([conv8u, conv1], axis=3)
        conv9 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(up9)
        conv9 = keras.layers.Conv2D(32, (3, 3), activation='relu', padding='SAME')(conv9)
        conv9 = tf.reshape(conv9, tf.shape(conv9))
    
        conv9u = tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv9)
        conv9u = tf.reshape(conv9u, tf.shape(conv9u))
        up10 = keras.layers.concatenate([conv9u, conv0], axis=3)
        conv10 = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='SAME')(up10)
        conv10 = keras.layers.Conv2D(16, (3, 3), activation='relu', padding='SAME')(conv10)
        conv10 = tf.reshape(conv10, tf.shape(conv10))

        conv11 = keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(conv10)
        conv11 = tf.reshape(conv11, tf.shape(conv11))

        model = keras.Model(inputs=[inputs], outputs=[conv11])

        model.compile(optimizer=optimizer(lr=lr), loss=loss_metric, metrics=metrics)
        return model

In [ ]:
# add layer
sample_width = 256
sample_height = 256

# tf.keras.optimizers.SGD
# tf.keras.optimizers.Adam

# dice_coef_loss
# tf.keras.losses.binary_crossentropy
# tf.keras.losses.BinaryCrossentropy()

model = get_model1(optimizer=tf.keras.optimizers.Adam,
                   loss_metric=dice_coef_loss, metrics=['AUC', dice_coef],  #dice_coef, 'binary_accuracy'
                   lr=1e-6
                  )
# 'AUC'

In [22]:
from keras import backend as K
import keras
# from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, Input
import tensorflow as tf

sample_width = 512
sample_height = 512
n_filters = 8

In [23]:
inputs = keras.Input((sample_width, sample_height, 1))
conv1 = keras.layers.Conv2D(n_filters*2, (3, 3), activation='relu', padding='SAME')(inputs)
conv1 = keras.layers.Conv2D(n_filters*2, (3, 3), activation='relu', padding='SAME')(conv1)
conv1 = tf.reshape(conv1, tf.shape(conv1))
pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
drop1 = keras.layers.Dropout(0.2)(pool1)

print(drop1.shape)

(None, 256, 256, 16)


In [24]:
conv2 = keras.layers.Conv2D(n_filters*4, (3, 3), activation='relu', padding='SAME')(drop1)
conv2 = keras.layers.Conv2D(n_filters*4, (3, 3), activation='relu', padding='SAME')(conv2)
conv2 = tf.reshape(conv2, tf.shape(conv2))
pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
drop2 = keras.layers.Dropout(0.2)(pool2)

print(drop2.shape)

(None, 128, 128, 32)


In [25]:
conv3 = keras.layers.Conv2D(n_filters*8, (3, 3), activation='relu', padding='SAME')(drop2)
conv3 = keras.layers.Conv2D(n_filters*8, (3, 3), activation='relu', padding='SAME')(conv3)
conv3 = tf.reshape(conv3, tf.shape(conv3))
pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
drop3 = keras.layers.Dropout(0.3)(pool3)

print(drop3.shape)

(None, 64, 64, 64)


In [26]:
conv4 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(drop3)
conv4 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(conv4)
conv4 = tf.reshape(conv4, tf.shape(conv4))
pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
drop4 = keras.layers.Dropout(0.3)(pool4)

print(drop4.shape)

(None, 32, 32, 128)


In [27]:
conv5 = keras.layers.Conv2D(n_filters*32, (3, 3), activation='relu', padding='SAME')(drop4)
conv5 = keras.layers.Conv2D(n_filters*32, (3, 3), activation='relu', padding='SAME')(conv5)
conv5 = tf.reshape(conv5, tf.shape(conv5))
    
print(conv5.shape)

(None, 32, 32, 256)


In [28]:
conv5u = tf.keras.layers.Conv2DTranspose(filters=n_filters*16, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv5)
conv5u = tf.reshape(conv5u, tf.shape(conv5u))
up6 = keras.layers.concatenate([conv5u, conv4], axis=3)
up6 = keras.layers.Dropout(0.2)(up6)
conv6 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(up6)
conv6 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(conv6)
conv6 = tf.reshape(conv6, tf.shape(conv6))
    
print(conv5u.shape, conv4.shape, "\n", conv6.shape)

(None, 64, 64, 128) (None, 64, 64, 128) 
 (None, 64, 64, 128)


In [29]:
conv5u = keras.layers.Conv2D(n_filters*16, 2, activation = 'relu', padding = 'same')(tf.keras.layers.UpSampling2D(size = (2,2))(conv5))
conv5u = tf.reshape(conv5u, tf.shape(conv5u))
up6 = keras.layers.concatenate([conv5u, conv4], axis=3)
up6 = keras.layers.Dropout(0.2)(up6)
conv6 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(up6)
conv6 = keras.layers.Conv2D(n_filters*16, (3, 3), activation='relu', padding='SAME')(conv6)
conv6 = tf.reshape(conv6, tf.shape(conv6))
print(conv5u.shape, conv4.shape, "\n", conv6.shape)

(None, 64, 64, 128) (None, 64, 64, 128) 
 (None, 64, 64, 128)


In [30]:
conv6u = tf.keras.layers.Conv2DTranspose(filters=n_filters*8, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv6)
conv6u = tf.reshape(conv6u, tf.shape(conv6u))
up7 = keras.layers.concatenate([conv6u, conv3], axis=3)
up7 = keras.layers.Dropout(0.2)(up7)
conv7 = keras.layers.Conv2D(n_filters*8, (3, 3), activation='relu', padding='SAME')(up7)
conv7 = keras.layers.Conv2D(n_filters*8, (3, 3), activation='relu', padding='SAME')(conv7)
conv7 = tf.reshape(conv7, tf.shape(conv7))
    
print(conv6u.shape, conv3.shape, "\n", conv7.shape)

(None, 128, 128, 64) (None, 128, 128, 64) 
 (None, 128, 128, 64)


In [31]:
conv7u = tf.keras.layers.Conv2DTranspose(filters=n_filters*4, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv7)
conv7u = tf.reshape(conv7u, tf.shape(conv7u))
up8 = keras.layers.concatenate([conv7u, conv2], axis=3)
up8 = keras.layers.Dropout(0.2)(up8)
conv8 = keras.layers.Conv2D(n_filters*4, (3, 3), activation='relu', padding='SAME')(up8)
conv8 = keras.layers.Conv2D(n_filters*4, (3, 3), activation='relu', padding='SAME')(conv8)
conv8 = tf.reshape(conv8, tf.shape(conv8))
    
print(conv7u.shape, conv2.shape, "\n", conv8.shape)

(None, 256, 256, 32) (None, 256, 256, 32) 
 (None, 256, 256, 32)


In [10]:
conv8u = tf.keras.layers.Conv2DTranspose(filters=n_filters*2, kernel_size=(2, 2), strides=(2, 2), padding='SAME')(conv8)
conv8u = tf.reshape(conv8u, tf.shape(conv8u))
up9 = keras.layers.concatenate([conv8u, conv1], axis=3)
up9 = keras.layers.Dropout(0.2)(up9)
conv9 = keras.layers.Conv2D(n_filters*2, (3, 3), activation='relu', padding='SAME')(up9)
conv9 = keras.layers.Conv2D(n_filters*2, (3, 3), activation='relu', padding='SAME')(conv9)
conv9 = tf.reshape(conv9, tf.shape(conv9))

print(conv8u.shape, conv1.shape, "\n", conv9.shape)

(None, 256, 256, 32) (None, 256, 256, 32) 
 (None, 256, 256, 32)


In [11]:
conv10 = keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)
conv10 = tf.reshape(conv10, tf.shape(conv10))
print(conv10.shape)

(None, 256, 256, 1)
